In [1]:
import torch as tr
import numpy as np
import itertools
from CSW import *

from matplotlib import pyplot as plt
%matplotlib inline

%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [2]:
rootdir = 'matchSEM/sweep1/'

# Analysis: CI in layer 2 vs 3

In [ ]:
# eval params
depthL = [2,3]
tonodesL = [(3,4),(5,6)]
eval_pathL = [
  [0,10,1,4,5],       
  [0,11,1,3,5],
  [0,11,2,4,6],
    ]

In [ ]:
# sweep params
stsizeL=[20,25,30,35,40]
lrL=[0.005,0.01,0.05,0.1,0.5]
bpmodeL=['story']

### load and plt ysm

In [3]:
def load_ysm(blocklen,bpmode,stsize,lr):
  fpath = rootdir + "model_data/ysm-blocklen%i-bpmode_%s-stsize%i-lr%.3f.npy"%(blocklen,bpmode,stsize,lr)
  return np.load(fpath)

In [9]:
def plt_ysm(ysm_i,ysm_b,pathidx=0):
  fig,axarr = plt.subplots(2,2,figsize=(14,6),sharey=True,sharex=True)
  for ib_idx,ysm_plt in enumerate([ysm_i,ysm_b]):
    for depthidx in [0,1]:
      ax = axarr[ib_idx,depthidx]
      depth = depthL[depthidx]
      tonodes = tonodesL[depthidx]
      ax.plot(ysm_plt[:,pathidx,:,depth,tonodes].mean(1).T)
      path = eval_pathL[pathidx]
      fromnode = path[depth]
      sch = path[1]
      ax.set_title('schema%i_from-%i_to-%ior%i-%s'%(sch,fromnode,*tonodes,['interleaved','blocked'][ib_idx]))
  return None


In [15]:
for bpmode,stsize,lr in itertools.product(bpmodeL,stsizeL,lrL):
  print(bpmode,stsize,lr)
  ysm_i = load_ysm(1,bpmode,stsize,lr)
  ysm_b = load_ysm(40,bpmode,stsize,lr)
  plt_ysm(ysm_i,ysm_b)
  fpath = "ysm-bpmode_%s-stsize%i-lr%.3f.png"%(bpmode,stsize,lr)
  plt.suptitle(fpath)
  plt.savefig(rootdir+'figures/'+fpath)
  plt.close('all')



story 20 0.005
story 20 0.01
story 20 0.05
story 20 0.1
story 20 0.5
story 25 0.005
story 25 0.01
story 25 0.05
story 25 0.1
story 25 0.5
story 30 0.005
story 30 0.01
story 30 0.05
story 30 0.1
story 30 0.5
story 35 0.005
story 35 0.01
story 35 0.05
story 35 0.1
story 35 0.5
story 40 0.005
story 40 0.01
story 40 0.05
story 40 0.1
story 40 0.5


### load and plt accuracy

In [38]:
def load_acc(blocklen,bpmode,stsize,lr):
  fpath = rootdir + "model_data/acc-blocklen%i-bpmode_%s-stsize%i-lr%.3f.npy"%(blocklen,bpmode,stsize,lr)
  return np.load(fpath)

def plt_acc(acc_i,acc_b,pathidx=0):
  plt.plot(acc_i)
  plt.plot(acc_b)
  plt.ylim(0,1.05)
  plt.axhline(0.5,c='k',ls='--')
  return None

In [41]:
for bpmode,stsize,lr in itertools.product(bpmodeL,stsizeL,lrL):
  print(bpmode,stsize,lr)
  plt.figure(figsize=(10,6))

  acc_i = load_acc(1,bpmode,stsize,lr).mean(0)
  acc_b = load_acc(40,bpmode,stsize,lr).mean(0)
  plt_acc(acc_i,acc_b,pathidx=0)
  fpath = "acc-bpmode_%s-stsize%i-lr%.3f.png"%(bpmode,stsize,lr)
  plt.suptitle(fpath)
  plt.savefig(rootdir+'figures/'+fpath)
  plt.close('all')



story 20 0.005
story 20 0.01
story 20 0.05
story 20 0.1
story 20 0.5
story 25 0.005
story 25 0.01
story 25 0.05
story 25 0.1
story 25 0.5
story 30 0.005
story 30 0.01
story 30 0.05
story 30 0.1
story 30 0.5
story 35 0.005
story 35 0.01
story 35 0.05
story 35 0.1
story 35 0.5
story 40 0.005
story 40 0.01
story 40 0.05
story 40 0.1
story 40 0.5


In [ ]:

  fpath = "acc-bpmode_%s-stsize%i-lr%.3f.png"%(bpmode,stsize,lr)
  plt.suptitle(fpath)
  plt.savefig(rootdir+'figures/'+fpath)
  plt.close('all')



# sweep code

In [14]:
import sys
import torch as tr
import numpy as np
from CSW import *

rootdir = 'matchSEM/sweep1/'

## sweep vars
block_len0 = int(sys.argv[1])
stsize = int(sys.argv[2])
learn_rate = float(sys.argv[3])
bpmode = sys.argv[4]

print(block_len0,stsize,learn_rate,bpmode) 

# fix vars
nnets = 20
neps = 200

## task init
taskL = [CSWTask(1.0),CSWTask(0.0)]

# eval paths
eval_pathL = [
  [0,10,1,4,5],       
  [0,11,1,3,5],
  [0,11,2,4,6],
  ]

# format for nn input
xeval = taskL[0].format_Xeval(eval_pathL)
xeval = tr.tensor(xeval)

# eval array
tdim,sm_dim=5,12
ysm = -np.ones([nnets,3,neps,tdim,sm_dim])
acc = -np.ones([nnets,neps])

## train setup
softmax = lambda ulog: tr.softmax(ulog,-1)
lossop = tr.nn.CrossEntropyLoss()

# loop over seeds
for seed in range(nnets):
  # init net
  net = CSWNet(stsize,seed)
  optiop = tr.optim.Adam(net.parameters(), lr=learn_rate)
  # init loop vars
  task_int = 0
  block_len = block_len0 
  # train loop
  for ep in range(neps):
    # curriculum select graph
    if ep == 160:
      block_len = 1
    if ep%block_len==0:
      task_int = (task_int+1)%2
      task = taskL[task_int]
      filler_id = 10+task_int 
    # forward prop  
    path = task.sample_path()
    xtrain,ytrain = task.dataset_onestory_with_marker(path=path,filler_id=filler_id,depth=1)
    yh = net(xtrain) # (time,smunits)
    # accuracy on train data
    yh_sm = softmax(yh).detach().numpy()
    yt = ytrain.detach().numpy().squeeze()
    acc[seed,ep] = np.mean(np.equal(np.argmax(yh_sm[2:4],1),yt[2:4]))
    ## eval dataset
    for idx,xev in enumerate(xeval):
      ysm_t = softmax(net(xev)).detach().numpy()
      ysm[seed,idx,ep] = ysm_t
    if bpmode=='scene':
      loss = 0
      for tstep in range(len(xtrain)):
        loss = lossop(yh[tstep].unsqueeze(0),ytrain[tstep])
        optiop.zero_grad()
        loss.backward(retain_graph=True)
        optiop.step()
    elif bpmode=='story':
      loss = 0
      for tstep in range(len(xtrain)):
        loss += lossop(yh[tstep].unsqueeze(0),ytrain[tstep])
      optiop.zero_grad()
      loss.backward(retain_graph=True)
      optiop.step()
    else: 
      assert False


np.save(rootdir+'model_data/ysm-blocklen%i-bpmode_%s-stsize%i-lr%.3f.npy'%(block_len0,bpmode,stsize,learn_rate),ysm)
np.save(rootdir+'model_data/acc-blocklen%i-bpmode_%s-stsize%i-lr%.3f.npy'%(block_len0,bpmode,stsize,learn_rate),acc)

